In [1]:
import importlib

import training
import utils
import training_constants as tc
from preprocessing import encode_labels, load_data, split_train_test_data

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
importlib.reload(training)
importlib.reload(utils)

<module 'utils' from '/Users/patricia/Documents/code/python/behavior-detection/scripts/utils.py'>

In [3]:
X, y = load_data()
y.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5525 entries, 0 to 5524
Columns: 336 entries, id_log to misc_EqSec_OpFrac_MMC_MtNumerador_total
dtypes: float64(82), int64(253), object(1)
memory usage: 14.2+ MB
<class 'pandas.core.series.Series'>
RangeIndex: 5525 entries, 0 to 5524
Series name: comportamento
Non-Null Count  Dtype 
--------------  ----- 
5525 non-null   object
dtypes: object(1)
memory usage: 43.3+ KB


0    ON TASK
1    ON TASK
2    ON TASK
3    ON TASK
4    ON TASK
Name: comportamento, dtype: object

In [4]:
X_train, X_test, y_train, y_test = split_train_test_data(X, y)


Tamanho do conjunto de treino: 3867
Tamanho do conjunto de teste: 1658


In [5]:
y_train, label_encoder = encode_labels(y_train)
y_test = label_encoder.transform(y_test)


In [6]:

importlib.reload(training)
trained_models = training.train_model(X_train, y_train, tc.GRID_SEARCH)


Training and evaluating Random Forest with GridSearch:
Grid search for RandomForestClassifier()
Param grid: {'classifier__n_estimators': [10, 50, 100, 200], 'classifier__max_depth': [None, 10, 20, 30], 'classifier__min_samples_split': [2, 5, 10], 'classifier__min_samples_leaf': [1, 2, 4]}
X_train shape: (3867, 336)
y_train shape: (3867,)
Fitting 5 folds for each of 144 candidates, totalling 720 fits
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled o

KeyError: 'Gradient Boosting'

In [ ]:
import evaluation
import importlib

importlib.reload(evaluation)
dirpath = "/Users/patricia/Documents/code/python/behavior-detection/output/"

reports = evaluation.generate_reports(trained_models, X_train, y_train, X_test, y_test)
print(evaluation.print_reports(reports, dirpath))
evaluation.save_reports_to_csv(reports, dirpath)